# Model Generation

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import configparser
import pandas as pd

import torch
import pytorch_lightning as pl

from pynas.core.population import Population
from datasets.RawClassifier.loader import RawClassifierDataModule

# Define dataset module
root_dir = '/Data_large/marine/PythonProjects/OtherProjects/lpl-PyNas/data/RawClassifier'
dm = RawClassifierDataModule(root_dir, batch_size=4, num_workers=2, transform=None)

config = configparser.ConfigParser()
config.read('config.ini')
def setting():
    pd.set_option('display.max_colwidth', None)
    # Logging
    logs_directory = str(config['GA']['logs_dir_GA'])
    # Torch stuff
    seed = config.getint(section='Computation', option='seed')
    pl.seed_everything(seed=seed, workers=True)  # For reproducibility
    torch.set_float32_matmul_precision("medium")  # to make lightning happy
setting()

Seed set to 42


In [3]:
# Model parameters
max_layers = 3
max_iter = int(config['GA']['max_iterations'])
# GA parameters
n_individuals = int(config['GA']['population_size'])
mating_pool_cutoff = float(config['GA']['mating_pool_cutoff'])
mutation_probability = float(config['GA']['mutation_probability'])

pop = Population(n_individuals=20, max_layers=max_layers, dm=dm, max_parameters=400_000)

In [4]:
pop.initial_poll()

Generating Population: 100%|██████████| 20/20 [00:23<00:00,  1.19s/it]


In [ ]:
pop.train_generation(task='classification', lr=0.001, epochs=15, batch_size=32)

In [ ]:
pop.evolve(mating_pool_cutoff=mating_pool_cutoff, mutation_probability=0.85, k_best=1, n_random=3)

In [ ]:
pop.load_dataframe(9)


# To Be Rewritten with Bash Logic:

This part of the code train the whole population one individual at the time and can be performed on single individuals in parallel. Using bash we can run parallel training. 

# Inference

Using the evaluated and saved model. We use the traced pytroch model (.pt) to load and execute inference.

In [ ]:
# Load the saved TorchScript model and test with a dummy input.
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

save_path = "model_and_architecture.pt"
loaded_model = torch.jit.load(save_path, map_location=device)
loaded_model.eval()

# Ensure input is moved to the correct device
example_input = torch.randn(1, *dm.input_shape).to(device)
example_input = example_input.to(device)

with torch.no_grad():
    output = loaded_model(example_input)
print("Output from the loaded model:", output)